In [1]:
import pandas as pd

data = pd.read_csv('Uhren Data Export single.tsv', sep='\t', header=0, index_col=0)
columns = data.columns
print (columns)

# data = data[[]]

C:\Users\fal45830\AppData\Local\Temp\ipykernel_9284\2124626165.py:3: DtypeWarning: Columns (37,38,65,66,67,68,69,70,72,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Uhren Data Export single.tsv', sep='\t', header=0, index_col=0)


Index(['Computer timestamp', 'Sensor', 'Project name', 'Export date',
       'Participant name', 'Recording name', 'Recording date',
       'Recording date UTC', 'Recording start time',
       'Recording start time UTC',
       ...
       'Client area position X (DACSpx)', 'Client area position Y (DACSpx)',
       'Viewport position X', 'Viewport position Y', 'Viewport width',
       'Viewport height', 'Full page width', 'Full page height',
       'Mouse position X', 'Mouse position Y'],
      dtype='object', length=206)
